In [2]:
!pip install IMDbPY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 4.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests
from imdb import IMDb, IMDbError

# Adding Artist Images to Songs

In [ ]:
song_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/song_final.csv")

In [ ]:
song_df.head()

,id,title,tag,artist,year,views,lyrics
0,mu_235729,Rap God,rap,Eminem,2013,17575634,look was gonna easy you not hurt your feelings...
1,mu_5832126,WAP,rap,Cardi B,2020,16003444,whores this house theres some whores this hous...
2,mu_2949128,Shape of You,pop,Ed Sheeran,2017,14569727,the club isnt the best place find lover the ba...
3,mu_3039923,HUMBLE.,rap,Kendrick Lamar,2017,11181199,nobody pray for been that day for way yeah yea...
4,mu_727466,The Hills,rb,The Weeknd,2015,9291775,yeah yeah yeah your man the road doin promo yo...


In [ ]:
song_df.shape

(200000, 7)

In [ ]:
# sets personal info from spotify developer
CLIENT_ID = "5f976fc652814c9490e9989f6f75f3d9"
CLIENT_SECRET = "2f92409be1124ee6ba6e21a10348fb27"

# authenticates my app
auth_url = "https://accounts.spotify.com/api/token"

auth_response = requests.post(auth_url, {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
})

# converts the response to JSON
auth_response_data = auth_response.json()

# saves the access token
access_token = auth_response_data["access_token"]

In [ ]:
# finds one artist's Spotify ID
artist_name = "Taylor Swift"
search_url = "https://api.spotify.com/v1/search"

headers = {"Authorization": f"Bearer {access_token}"}
params = {
    "q": artist_name,
    "type": "artist",
    "limit": 1
}

response = requests.get(search_url, headers=headers, params=params)
search_results = response.json()

# extracts artist ID
artist_id = search_results["artists"]["items"][0]["id"] if search_results["artists"]["items"] else None
print(artist_id)

06HL4z0CvFAxyc27GXpf02


In [ ]:
# makes a request for the artist's data
artist_url = f"https://api.spotify.com/v1/artists/{artist_id}"

response = requests.get(artist_url, headers=headers)
artist_data = response.json()

In [ ]:
# extracts image URL
image_url = artist_data["images"][0]["url"] if artist_data["images"] else None
print(image_url)

https://i.scdn.co/image/ab6761610000e5ebe672b5f553298dcdccb0e676


In [ ]:
# removes duplicated artist names
artist_names = list(set(list(song_df["artist"])))
print(len(artist_names))

33154


In [ ]:
# separates artists into 7 groups to avoid hitting the access limit
artist_names1 = artist_names[:5000]
artist_names2 = artist_names[5000:10000]
artist_names3 = artist_names[10000:15000]
artist_names4 = artist_names[15000:20000]
artist_names5 = artist_names[20000:25000]
artist_names6 = artist_names[25000:30000]
artist_names7 = artist_names[30000:]

In [ ]:
# creates function to get artist IDs
def get_artist_ids(artist_list):

  # initializes dictionary
  artist_ids = {}

  # iterates through the artist name list
  for idx, artist in enumerate(artist_list):
    params = {
      "q": artist,
      "type": "artist",
      "limit": 1
    }
    response = requests.get(search_url, headers=headers, params=params)

    if response.status_code == 200:
      # only decode JSON if the request was successful
      search_results = response.json()

      # extracts artist ID and add it to the dictionary
      artist_id = search_results["artists"]["items"][0]["id"] if search_results["artists"]["items"] else None
      artist_ids[artist] = artist_id

    else:
      # handles errors or unexpected status codes
      print(f"Error with status code {response.status_code} for artist: {artist} (idx: {idx})")
      artist_ids[artist] = None

    if idx % 1000 == 0:
      print(f"Processed {idx} artists...")

  return artist_ids

In [ ]:
# creates a function to extract image URLs
def get_image_urls(artist_list, artist_ids):

  # initializes dictionary
  image_urls={}

  # iterates through the artist name list
  for idx, name in enumerate(artist_list):

    # gets artist ID from the dictionary and sets the URL
    id = artist_ids[name]
    artist_url = f"https://api.spotify.com/v1/artists/{id}"

    response = requests.get(artist_url, headers=headers)

    if response.status_code == 200:
      # only decode JSON if the request was successful
      artist_data = response.json()

      # extracts image URL and add it to the dictionary
      image_url = artist_data["images"][0]["url"] if artist_data["images"] else None
      image_urls[name] = image_url

    else:
      # if there is no corresponding image, adds "no image" to the list and returns to the loop
      # we can handle this on the client side by converting it to the default image
      print(f"Error with status code {response.status_code} for artist: {name} (idx: {idx})")
      image_urls[name] = "no image"

    if idx % 1000 == 0:
        print(f"Processed {idx} artists...")

  return image_urls

In [ ]:
artist_ids1 = get_artist_ids(artist_names1)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url1 = get_image_urls(artist_names1, artist_ids1)

Processed 0 artists...
Error with status code 400 for artist: Turn the Music Louder (Rumble) Lyrics - KDA (idx: 118)
Error with status code 400 for artist: She's Kinda Hot Lyrics - 5 Seconds of Summer (idx: 141)
Processed 1000 artists...
Error with status code 400 for artist: BecauseThe.Com (idx: 1016)
Error with status code 400 for artist: On Me Lyrics - The Game (idx: 1416)
Error with status code 400 for artist: PARTYNEXTDOOR & Rihanna (idx: 1808)
Processed 2000 artists...
Error with status code 400 for artist: Only The Family, Lil Durk & King Von (idx: 2123)
Error with status code 400 for artist: Pegboard Nerds & Grabbitz (idx: 2276)
Processed 3000 artists...
Error with status code 400 for artist: for KING & COUNTRY & Dolly Parton (idx: 3870)
Processed 4000 artists...
Error with status code 400 for artist: Ran off on da Plug Twice (Ritz Carlton) Lyrics - Plies (idx: 4067)
Error with status code 400 for artist:   (Alla Pugacheva) & 50 Cent (idx: 4165)
Error with status code 400 for a

In [ ]:
artist_url1_df = pd.DataFrame(artist_url1.items(), columns=["artist", "image"])

In [ ]:
artist_url1_df.head()

,artist,image
0,Xan Griffin,https://i.scdn.co/image/ab6761610000e5ebe2bda6...
1,Warrant,https://i.scdn.co/image/ab6761610000e5ebca9337...
2,Mr Blobby,https://i.scdn.co/image/ab6761610000e5eba1393c...
3,Reuben And The Dark,https://i.scdn.co/image/ab6761610000e5eb6895c4...
4,Cloud 5,https://i.scdn.co/image/ab67616d0000b273e7ba99...


In [ ]:
artist_url1_df.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/artist_url1.csv")

In [ ]:
def make_csv(artist_url, num):
  artist_url_df = pd.DataFrame(artist_url.items(), columns=["artist", "image"])
  artist_url_df.to_csv(f"/content/drive/MyDrive/Multimedia Recommendation/splitted data/artist_url{num}.csv")

In [ ]:
artist_ids2 = get_artist_ids(artist_names2)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url2 = get_image_urls(artist_names2, artist_ids2)

Processed 0 artists...
Error with status code 400 for artist: Years & Years & MNEK (idx: 140)
Processed 1000 artists...
Error with status code 400 for artist: Best of Wives and Best of Women Lyrics - Lin-Manuel Miranda & Phillipa Soo (idx: 1751)
Processed 2000 artists...
Error with status code 400 for artist: Rejjie Snow, Snoh Aalegra & Cam O'bi (idx: 2758)
Error with status code 400 for artist: Caged Bird Lyrics - J. Cole (idx: 2953)
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
make_csv(artist_url2, 2)

In [ ]:
artist_ids3 = get_artist_ids(artist_names3)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url3 = get_image_urls(artist_names3, artist_ids3)

Processed 0 artists...
Error with status code 400 for artist: Only The Family, Doodie Lo & King Von (idx: 117)
Error with status code 400 for artist: Love in the Dark Lyrics - Adele (idx: 131)
Error with status code 400 for artist: I Want to Write You a Song Lyrics - One Direction (idx: 294)
Processed 1000 artists...
Error with status code 400 for artist: Felly, Kota the Friend & Monte Booker (idx: 1131)
Error with status code 400 for artist: The Stop the Violence Movement (idx: 1956)
Processed 2000 artists...
Error with status code 400 for artist: Too Indecent (idx: 2334)
Processed 3000 artists...
Error with status code 400 for artist: Kanye West, JAY-Z & Big Sean (idx: 3575)
Error with status code 400 for artist: Only The Family & MK (idx: 3606)
Processed 4000 artists...
Error with status code 400 for artist: Pg.99 (idx: 4464)


In [ ]:
make_csv(artist_url3, 3)

In [ ]:
artist_ids4 = get_artist_ids(artist_names4)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url4 = get_image_urls(artist_names4, artist_ids4)

Processed 0 artists...
Error with status code 400 for artist: Only The Family, Lil Durk & Doodie Lo (idx: 13)
Error with status code 400 for artist: Why Don't We & Macklemore (idx: 380)
Error with status code 400 for artist: Piss on Your Grave Lyrics - Travis Scott (idx: 385)
Error with status code 400 for artist: You Mine Lyrics - DJ Khaled (idx: 753)
Error with status code 400 for artist: Summer Walker & PARTYNEXTDOOR (idx: 880)
Processed 1000 artists...
Error with status code 400 for artist: Hit the Ground Lyrics - Justin Bieber (idx: 1286)
Error with status code 400 for artist: Talk About It Lyrics - Dr. Dre (idx: 1319)
Error with status code 400 for artist: My Family Pies (idx: 1583)
Processed 2000 artists...
Error with status code 400 for artist: H34567ujkhgd (idx: 2010)
Error with status code 400 for artist: H-Bass and the Kosby Kids (idx: 2227)
Error with status code 400 for artist: Drive Lyrics - Oh Wonder (idx: 2318)
Error with status code 400 for artist: Beyonc & Kendrick La

In [ ]:
make_csv(artist_url4, 4)

In [ ]:
artist_ids5 = get_artist_ids(artist_names5)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url5 = get_image_urls(artist_names5, artist_ids5)
make_csv(artist_url5, 5)

Processed 0 artists...
Error with status code 400 for artist: Hit The Quan Lyrics - iLoveMemphis (idx: 396)
Error with status code 400 for artist: All In It Lyrics - Justin Bieber (idx: 553)
Error with status code 400 for artist: Only The Family, Lil Durk & Chief Wuk (idx: 651)
Error with status code 400 for artist: No Rome, Charli XCX & The 1975 (idx: 653)
Error with status code 400 for artist: Past Life Lyrics - Tame Impala (idx: 764)
Error with status code 400 for artist: Said The Sky, ILLENIUM, & Chelsea Cutler (idx: 765)
Error with status code 400 for artist: Beyonc, JAY-Z & Childish Gambino (idx: 788)
Error with status code 400 for artist: Ariana Grande, Miley Cyrus & Lana Del Rey (idx: 789)
Processed 1000 artists...
Error with status code 400 for artist: That Would Be Enough Lyrics - Lin-Manuel Miranda & Phillipa Soo (idx: 1952)
Processed 2000 artists...
Error with status code 400 for artist: Wiped Out! Lyrics - The Neighbourhood (idx: 2141)
Error with status code 400 for artist

In [ ]:
artist_ids6 = get_artist_ids(artist_names6)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...
Processed 4000 artists...


In [ ]:
artist_url6 = get_image_urls(artist_names6, artist_ids6)
make_csv(artist_url6, 6)

Processed 0 artists...
Error with status code 400 for artist: Apple Jack, Rainbow Dash, Pinkie Pie, Rarity & Fluttershy (idx: 779)
Error with status code 400 for artist: C00lb00bs (idx: 888)
Processed 1000 artists...
Error with status code 400 for artist: Fac88 (idx: 1073)
Error with status code 400 for artist: I Serve the Base Lyrics - Future (idx: 1192)
Error with status code 400 for artist: CB (USA) (idx: 1280)
Error with status code 400 for artist: Picture Me Rollin' Lyrics - Chris Brown (idx: 1869)
Processed 2000 artists...
Error with status code 400 for artist: All I Ask Lyrics - Adele (idx: 2038)
Error with status code 400 for artist: I Miss You Lyrics - Adele (idx: 2116)
Error with status code 400 for artist: Simple Plan & State Champs (idx: 2166)
Error with status code 400 for artist: I Smell Panties (idx: 2529)
Error with status code 400 for artist: One Pound Fish Man (idx: 2867)
Error with status code 400 for artist: Seyettttttttttt (idx: 2975)
Processed 3000 artists...
Erro

In [ ]:
artist_ids7 = get_artist_ids(artist_names7)

Processed 0 artists...
Processed 1000 artists...
Processed 2000 artists...
Processed 3000 artists...


In [ ]:
artist_url7 = get_image_urls(artist_names7, artist_ids7)
make_csv(artist_url7, 7)

Processed 0 artists...
Processed 1000 artists...
Error with status code 400 for artist: Miley Cyrus, Swae Lee & Mike WiLL Made-It (idx: 1149)
Error with status code 400 for artist: Lin-Manuel Miranda, Daveed Diggs, Leslie Odom Jr. & Okieriete Onaodowan (idx: 1277)
Error with status code 400 for artist: Kno the Meaning Lyrics - Future (idx: 1783)
Error with status code 400 for artist: Dave & Snoh Aalegra (idx: 1825)
Processed 2000 artists...
Error with status code 400 for artist: Said The Sky & Kerli (idx: 2144)
Error with status code 400 for artist: 88rising, Rich Brian, Yung Bans, Yung Pinch, Higher Brothers & Don Krez (idx: 2288)
Error with status code 400 for artist: Blue Ivy, SAINt JHN, Beyonc & Wizkid (idx: 2678)
Error with status code 400 for artist: Feed Me & Crystal Fighters (idx: 2724)
Processed 3000 artists...


In [ ]:
# combine all image URLs
artist_url_all = {**artist_url1, **artist_url2, **artist_url3, **artist_url4, **artist_url5, **artist_url6, **artist_url7}

In [ ]:
artist_url_all

{'Xan Griffin': 'https://i.scdn.co/image/ab6761610000e5ebe2bda6715fd23dae9c7fd721',
 'Warrant': 'https://i.scdn.co/image/ab6761610000e5ebca93371adcc6ab1e92497687',
 'Mr Blobby': 'https://i.scdn.co/image/ab6761610000e5eba1393cad16da2cc29d6b0e3c',
 'Reuben And The Dark': 'https://i.scdn.co/image/ab6761610000e5eb6895c4ecc59c0dc911c33c3f',
 'Cloud 5': 'https://i.scdn.co/image/ab67616d0000b273e7ba99ba65c23c17d64f43c0',
 'Stop Light Observations': 'https://i.scdn.co/image/ab6761610000e5ebe3ef5e8c96f457b1fe29b02b',
 'Slatt Zy & Hotboii': 'https://i.scdn.co/image/ab6761610000e5eb92790922e37b37d15bda35b8',
 'Blue': 'https://i.scdn.co/image/ab6761610000e5eb9ca66f0901a76a7ac29c04e9',
 'Zak Abel': 'https://i.scdn.co/image/ab6761610000e5ebc32bd91cc0061f54a9529b73',
 'Dubkonscious': 'https://i.scdn.co/image/ab67616d0000b27326fdbc0a6942559121d7a78a',
 '2nd Exit': 'https://i.scdn.co/image/ab6761610000e5ebcc3d0b1cef89da88b4e0a93c',
 'Feint': 'https://i.scdn.co/image/ab6761610000e5eb7ce019cab4cdc2bc93c2

In [ ]:
# creates a new column with image url data
song_df["image"] = song_df["artist"].apply(lambda x: artist_url_all[x])

In [ ]:
song_df.head()

,id,title,tag,artist,year,views,lyrics,image
0,mu_235729,Rap God,rap,Eminem,2013,17575634,look was gonna easy you not hurt your feelings...,https://i.scdn.co/image/ab6761610000e5eba00b11...
1,mu_5832126,WAP,rap,Cardi B,2020,16003444,whores this house theres some whores this hous...,https://i.scdn.co/image/ab6761610000e5eb4e8a7e...
2,mu_2949128,Shape of You,pop,Ed Sheeran,2017,14569727,the club isnt the best place find lover the ba...,https://i.scdn.co/image/ab6761610000e5eb3bcef8...
3,mu_3039923,HUMBLE.,rap,Kendrick Lamar,2017,11181199,nobody pray for been that day for way yeah yea...,https://i.scdn.co/image/ab6761610000e5eb437b9e...
4,mu_727466,The Hills,rb,The Weeknd,2015,9291775,yeah yeah yeah your man the road doin promo yo...,https://i.scdn.co/image/ab6761610000e5eb214f3c...


In [ ]:
song_df.shape

(200000, 8)

In [ ]:
song_df["image"].isnull().sum()

3081

In [ ]:
song_df_fillna = song_df.fillna("default")

In [ ]:
song_df_fillna["image"].isnull().sum()

0

In [ ]:
# export to csv
song_df_fillna.to_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/song_final_link.csv", index=False)

In [ ]:
song_image_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/song_final_link.csv")

In [ ]:
song_image_df.head(10)

,Unnamed: 0,id,title,tag,artist,year,views,lyrics,image
0,0,mu_235729,Rap God,rap,Eminem,2013,17575634,look was gonna easy you not hurt your feelings...,https://i.scdn.co/image/ab6761610000e5eba00b11...
1,1,mu_5832126,WAP,rap,Cardi B,2020,16003444,whores this house theres some whores this hous...,https://i.scdn.co/image/ab6761610000e5eb4e8a7e...
2,2,mu_2949128,Shape of You,pop,Ed Sheeran,2017,14569727,the club isnt the best place find lover the ba...,https://i.scdn.co/image/ab6761610000e5eb3bcef8...
3,3,mu_3039923,HUMBLE.,rap,Kendrick Lamar,2017,11181199,nobody pray for been that day for way yeah yea...,https://i.scdn.co/image/ab6761610000e5eb437b9e...
4,4,mu_727466,The Hills,rb,The Weeknd,2015,9291775,yeah yeah yeah your man the road doin promo yo...,https://i.scdn.co/image/ab6761610000e5eb214f3c...
5,5,mu_1063,Bohemian Rhapsody,rock,Queen,1975,9247817,this the real life this just fantasy caught la...,https://i.scdn.co/image/b040846ceba13c3e9c125d...
6,6,mu_2845980,Bad and Boujee,rap,Migos,2016,9168421,you know young rich niggas you know somethin a...,https://i.scdn.co/image/ab6761610000e5ebf4593f...
7,7,mu_3315890,Gods Plan,rap,Drake,2018,9107059,and they wishin and wishin and wishin and wish...,https://i.scdn.co/image/ab6761610000e5eb429338...
8,8,mu_3876994,SICKO MODE,rap,Travis Scott,2018,9078511,astro yeah sun down freezin cold thats how alr...,https://i.scdn.co/image/ab6761610000e5eb19c279...
9,9,mu_4063065,​thank u next,pop,Ariana Grande,2018,9072131,thought end with sean but wasnt match wrote so...,https://i.scdn.co/image/ab6761610000e5eb40b5c0...


# Adding Poster Images to TV Shows

In [4]:
tv_df = pd.read_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/tv_final.csv")

In [5]:
tv_df.head()

,id,Series Title,Release Year,End Year,Runtime,Rating,Synopsis
0,tv_0,Wednesday,2022,,45,8.2,follows wednesday addams years student when sh...
1,tv_1,Yellowstone,2018,,60,8.7,ranching family montana faces off against othe...
2,tv_2,The White Lotus,2021,2023,60,7.9,set tropical resort follows the exploits vario...
3,tv_3,1923,2022,2023,60,8.6,the duttons face new set challenges the early ...
4,tv_4,Jack Ryan,2018,,60,8.0,upandcoming cia analyst jack ryan thrust into ...


In [6]:
tv_df.shape

(10003, 7)

In [7]:
tv_df0 = tv_df[:500]
tv_df1 = tv_df[500:1000]
tv_df2 = tv_df[1000:1500]
tv_df3 = tv_df[1500:2000]
tv_df4 = tv_df[2000:2500]
tv_df5 = tv_df[2500:3000]
tv_df6 = tv_df[3000:3500]
tv_df7 = tv_df[3500:4000]
tv_df8 = tv_df[4000:4500]
tv_df9 = tv_df[4500:5000]
tv_df10 = tv_df[5000:5500]
tv_df11 = tv_df[5500:6000]
tv_df12 = tv_df[6000:6500]
tv_df13 = tv_df[6500:7000]
tv_df14 = tv_df[7000:7500]
tv_df15 = tv_df[7500:8000]
tv_df16 = tv_df[8000:8500]
tv_df17 = tv_df[8500:9000]
tv_df18 = tv_df[9000:9500]
tv_df19 = tv_df[9500:]

In [8]:
# creates an instance of the IMDb class
imdb = IMDb()

# searches for the TV series by title
series_name = "The OA"
search_result = imdb.search_movie(series_name)[0].movieID

# gets series data
series_data = imdb.get_movie(search_result)

# gets poster url from the data
poster_url = series_data["full-size cover url"]
print(poster_url)

https://m.media-amazon.com/images/M/MV5BMTY5OTkwNDkzOF5BMl5BanBnXkFtZTgwMDEyNzI1NzM@.jpg


In [9]:
def get_poster_url(dataset):
    imdb = IMDb()
    series_names = list(dataset["Series Title"])
    poster_urls = []

    for idx, name in enumerate(series_names):
        try:
            # search for the movie and get the first result
            search_result = imdb.search_movie(name)[0].movieID

            # get movie details using the movie ID from the first search result
            series_data = imdb.get_movie(search_result)

            # check if the poster URL is available
            if "full-size cover url" in series_data.keys():
                poster_urls.append(series_data["full-size cover url"])

            else:
                print(f"Index {idx}: Poster URL not found")
                poster_urls.append("default")

            if idx % 100 == 0:
                print(f"Processed {idx} shows...")

        except IMDbError as e:
            print(f"IMDb error occurred while processing index {idx}: {e}")
            poster_urls.append("default")

        except TimeoutError:
            print(f"Timeout error occurred while processing index {idx}. Retrying...")
            # optionally retry the fetch operation, or simply append 'default'
            poster_urls.append("default")

        except Exception as e:
            print(f"An error occurred while processing index {idx}: {e}")
            poster_urls.append("default")

    return poster_urls

In [ ]:
poster_urls0 = get_poster_url(tv_df0)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Processed 300 shows...
Processed 400 shows...


In [ ]:
print(poster_urls0)

['https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https://m.media-amazon.com/images/M/MV5BM2ZmMjEyZmYtOGM4YS00YTNhLWE3ZDMtNzQxM2RhNjBlODIyXkEyXkFqcGdeQXVyMTUzMTg2ODkz.jpg', 'https:

In [ ]:
len(poster_urls0)

500

In [ ]:
tv_df0["image"] = poster_urls0
tv_df0.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url0.csv", index=False)

<ipython-input-123-09d2619e30e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df0["image"] = poster_urls0


In [ ]:
tv_df1.head()

,id,Series Title,Release Year,End Year,Runtime,Rating,Synopsis
500,tv_500,The OA,2016,2019,60,7.8,having gone missing seven years ago the previo...
501,tv_501,Truth Be Told,2019,,45,7.1,truecrime podcaster tries solve the mystery su...
502,tv_502,Let the Right One In,2022,,53,7.5,follows 12yearold girl who lives closedin life...
503,tv_503,Workin' Moms,2017,2023,30,7.8,four very different thirtysomething workingmot...
504,tv_504,Miraculous: Tales of Ladybug & Cat Noir,2015,,20,7.6,marinette and adrien two normal teens transfor...


In [ ]:
poster_urls1 = get_poster_url(tv_df1)
tv_df1["image"] = poster_urls1
tv_df1.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url1.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Index 136: Poster URL not found
Processed 200 shows...


2024-04-20 17:12:22,537 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0069658/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 237: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0069658/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


2024-04-20 17:13:08,361 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt2433738/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 242: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt2433738/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


...

2024-04-20 17:17:02,558 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0053494/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 279: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0053494/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
Processed 300 shows...
Processed 400 shows...
Index 444: Poster URL not found


2024-04-20 17:29:24,159 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0088580/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 461: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0088580/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}


<ipython-input-18-626c3aed313a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df1["image"] = poster_urls1


In [ ]:
poster_urls2 = get_poster_url(tv_df2)
tv_df2["image"] = poster_urls2
tv_df2.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url2.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Index 230: Poster URL not found


2024-04-20 17:49:30,976 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0068069/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 259: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0068069/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
Processed 300 shows...
Processed 400 shows...
An error occurred while processing index 488: list index out of range


<ipython-input-19-890332c9bb8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df2["image"] = poster_urls2


In [ ]:
poster_urls3 = get_poster_url(tv_df3)
tv_df3["image"] = poster_urls3
tv_df3.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url3.csv", index=False)

Processed 0 shows...
Index 11: Poster URL not found
Index 12: Poster URL not found
Processed 100 shows...
Processed 200 shows...


2024-04-20 18:19:30,717 CRITICAL [imdbpy] /usr/local/lib/python3.10/dist-packages/imdb/_exceptions.py:32: IMDbDataAccessError exception raised; args: ({'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0048845/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')},); kwds: {}
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/imdb/parser/http/__init__.py", line 233, in retrieve_unicode
    response = uopener.open(url)
  File "/usr/lib/python3.10/urllib/request.py", line 519, in open
    response = self._open(req, data)
  File "/usr/lib/python3.10/urllib/request.py", line 536, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/usr/lib/python3.10/urllib/request.py", line 496, in _call_chain
    result = func(*args)
  File "/usr/lib/python3.10/urllib/request.py", line 1391, in https_open
    return self.do_open(http.client.HTTPSConnec

IMDb error occurred while processing index 201: {'errcode': None, 'errmsg': 'None', 'url': 'https://www.imdb.com/title/tt0048845/reference', 'proxy': '', 'exception type': 'IOError', 'original exception': TimeoutError('The read operation timed out')}
Processed 300 shows...
Processed 400 shows...


<ipython-input-20-4d21c6747037>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df3["image"] = poster_urls3


In [ ]:
poster_urls4 = get_poster_url(tv_df4)
tv_df4["image"] = poster_urls4
tv_df4.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url4.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Index 204: Poster URL not found
Index 268: Poster URL not found
Processed 300 shows...
Processed 400 shows...


<ipython-input-21-7a1e30740631>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df4["image"] = poster_urls4


In [ ]:
poster_urls5 = get_poster_url(tv_df5)
tv_df5["image"] = poster_urls5
tv_df5.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url5.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Index 163: Poster URL not found
Processed 200 shows...
Processed 300 shows...
Processed 400 shows...
Index 477: Poster URL not found


<ipython-input-22-809ace8a8beb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df5["image"] = poster_urls5


In [ ]:
poster_urls6 = get_poster_url(tv_df6)
tv_df6["image"] = poster_urls6
tv_df6.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url6.csv", index=False)

In [ ]:
poster_urls7 = get_poster_url(tv_df7)
tv_df7["image"] = poster_urls7
tv_df7.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url7.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Index 256: Poster URL not found
Processed 300 shows...
Processed 400 shows...
Index 499: Poster URL not found


<ipython-input-24-a7dcf6bed39a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df7["image"] = poster_urls7


In [ ]:
poster_urls8 = get_poster_url(tv_df8)
tv_df8["image"] = poster_urls8
tv_df8.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url8.csv", index=False)

In [26]:
poster_urls9 = get_poster_url(tv_df9)
tv_df9["image"] = poster_urls9
tv_df9.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url9.csv", index=False)

Processed 0 shows...
Index 56: Poster URL not found
Index 64: Poster URL not found
Processed 100 shows...
Processed 200 shows...
Index 253: Poster URL not found
Index 282: Poster URL not found
Processed 300 shows...
Index 334: Poster URL not found
Index 381: Poster URL not found
Processed 400 shows...


<ipython-input-26-eb510723d6b7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df9["image"] = poster_urls9


In [27]:
poster_urls10 = get_poster_url(tv_df10)
tv_df10["image"] = poster_urls10
tv_df10.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url10.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Processed 300 shows...
Index 331: Poster URL not found
Index 343: Poster URL not found
Index 360: Poster URL not found
Processed 400 shows...
Index 424: Poster URL not found
Index 466: Poster URL not found


<ipython-input-27-3054e3911163>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df10["image"] = poster_urls10


In [28]:
poster_urls11 = get_poster_url(tv_df11)
tv_df11["image"] = poster_urls11
tv_df11.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url11.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Processed 200 shows...
Index 243: Poster URL not found
Processed 300 shows...
Index 352: Poster URL not found
Index 366: Poster URL not found
Index 390: Poster URL not found
Processed 400 shows...


<ipython-input-28-ba48d3a2e731>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df11["image"] = poster_urls11


In [10]:
poster_urls12 = get_poster_url(tv_df12)
tv_df12["image"] = poster_urls12
tv_df12.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url12.csv", index=False)

Processed 0 shows...
Index 54: Poster URL not found
Processed 100 shows...
Index 111: Poster URL not found
Processed 200 shows...
Processed 300 shows...
Index 364: Poster URL not found
Processed 400 shows...


<ipython-input-10-4b33e52022c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df12["image"] = poster_urls12


In [11]:
poster_urls13 = get_poster_url(tv_df13)
tv_df13["image"] = poster_urls13
tv_df13.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url13.csv", index=False)

Processed 0 shows...
Index 89: Poster URL not found
Processed 100 shows...
Processed 200 shows...
Processed 300 shows...
Processed 400 shows...


<ipython-input-11-dbe50bca5e80>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df13["image"] = poster_urls13


In [12]:
poster_urls14 = get_poster_url(tv_df14)
tv_df14["image"] = poster_urls14
tv_df14.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url14.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Index 188: Poster URL not found
Processed 200 shows...
Index 226: Poster URL not found
Processed 300 shows...
Processed 400 shows...
Index 452: Poster URL not found


<ipython-input-12-7924524cbeac>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df14["image"] = poster_urls14


In [13]:
poster_urls15 = get_poster_url(tv_df15)
tv_df15["image"] = poster_urls15
tv_df15.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url15.csv", index=False)

Processed 0 shows...
Processed 100 shows...
Index 104: Poster URL not found
Index 129: Poster URL not found
Processed 200 shows...
Index 259: Poster URL not found
Processed 300 shows...
Index 329: Poster URL not found
Index 395: Poster URL not found
Processed 400 shows...
Index 426: Poster URL not found


<ipython-input-13-f3369b83abd9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df15["image"] = poster_urls15


In [14]:
poster_urls16 = get_poster_url(tv_df16)
tv_df16["image"] = poster_urls16
tv_df16.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url16.csv", index=False)

Processed 0 shows...
Index 34: Poster URL not found
Index 77: Poster URL not found
Processed 100 shows...
Index 120: Poster URL not found
Index 180: Poster URL not found
Index 184: Poster URL not found
Processed 200 shows...
Processed 300 shows...
Index 350: Poster URL not found
Index 362: Poster URL not found
Processed 400 shows...
Index 406: Poster URL not found
Index 495: Poster URL not found


<ipython-input-14-63e15d392948>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df16["image"] = poster_urls16


In [15]:
poster_urls17 = get_poster_url(tv_df17)
tv_df17["image"] = poster_urls17
tv_df17.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url17.csv", index=False)

Processed 0 shows...
Index 69: Poster URL not found
Index 88: Poster URL not found
Processed 100 shows...
Index 174: Poster URL not found
Processed 200 shows...
Index 220: Poster URL not found
Processed 300 shows...
Index 327: Poster URL not found
Index 331: Poster URL not found
Index 368: Poster URL not found
Processed 400 shows...
Index 408: Poster URL not found
Index 495: Poster URL not found


<ipython-input-15-b26710f42ae5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df17["image"] = poster_urls17


In [16]:
poster_urls18 = get_poster_url(tv_df18)
tv_df18["image"] = poster_urls18
tv_df18.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url18.csv", index=False)

Processed 0 shows...
Index 1: Poster URL not found
Index 8: Poster URL not found
Index 17: Poster URL not found
Processed 100 shows...
Index 115: Poster URL not found
Index 121: Poster URL not found
Processed 200 shows...
Index 202: Poster URL not found
Index 241: Poster URL not found
Index 258: Poster URL not found
Index 271: Poster URL not found
Processed 300 shows...
An error occurred while processing index 302: list index out of range
Processed 400 shows...
Index 419: Poster URL not found
Index 441: Poster URL not found


<ipython-input-16-bdedf88febba>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df18["image"] = poster_urls18


In [17]:
poster_urls19 = get_poster_url(tv_df19)
tv_df19["image"] = poster_urls19
tv_df19.to_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url19.csv", index=False)

Processed 0 shows...
Index 45: Poster URL not found
Index 88: Poster URL not found
Processed 100 shows...
Index 148: Poster URL not found
Processed 200 shows...
Index 235: Poster URL not found
Index 292: Poster URL not found
Processed 300 shows...
An error occurred while processing index 337: list index out of range
Processed 400 shows...
Index 462: Poster URL not found
Processed 500 shows...


<ipython-input-17-56bee9f8ff3a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tv_df19["image"] = poster_urls19


In [19]:
tv_df_res = pd.read_csv("/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url0.csv")

In [20]:
tv_df_res.head()

,id,Series Title,Release Year,End Year,Runtime,Rating,Synopsis,image
0,tv_0,Wednesday,2022,,45.0,8.2,follows wednesday addams years student when sh...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
1,tv_1,Yellowstone,2018,,60.0,8.7,ranching family montana faces off against othe...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
2,tv_2,The White Lotus,2021,2023,60.0,7.9,set tropical resort follows the exploits vario...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
3,tv_3,1923,2022,2023,60.0,8.6,the duttons face new set challenges the early ...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...
4,tv_4,Jack Ryan,2018,,60.0,8.0,upandcoming cia analyst jack ryan thrust into ...,https://m.media-amazon.com/images/M/MV5BM2ZmMj...


In [21]:
len(tv_df_res)

500

In [22]:
for i in range (1, 20):
  df = pd.read_csv(f"/content/drive/MyDrive/Multimedia Recommendation/splitted data/poster_url{i}.csv")
  tv_df_res = pd.concat([tv_df_res, df], ignore_index=True)

In [23]:
tv_df_res.shape

(10003, 8)

In [24]:
tv_df_res.to_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/tv_final_link.csv", index=False)

In [25]:
tv_df_res.dtypes

id              object
Series Title    object
Release Year    object
End Year        object
Runtime         object
Rating          object
Synopsis        object
image           object
dtype: object

In [26]:
tv_df_res["image"].isna().sum()

0

In [27]:
tv_df_res["End Year"].isna().sum()

200

In [28]:
len(tv_df_res[tv_df_res["End Year"] == " "])

3647

In [29]:
tv_df_res["End Year"] = tv_df_res["End Year"].replace(" ", 2024)

In [30]:
len(tv_df_res[tv_df_res["End Year"] == " "])

0

In [32]:
len(tv_df_res[tv_df_res["Rating"] == "****"])

430

In [33]:
tv_df_res["Rating"] = tv_df_res["Rating"].replace("****", 0.0)

In [34]:
tv_df_res.to_csv("/content/drive/Shareddrives/5500 Database Project/datasets/Final Final Datasets/tv_final_link.csv", index=False)